# Add the necessary lines to the ini's

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# import bilby as bb

mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",  # or "sans-serif" or "monospace"
    "font.serif": ["Computer Modern Roman"],
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10
})

plt.style.use("seaborn-v0_8-paper")

In [5]:
def makeLabel(batch,item,spin):
    """
    Label should follow the following structure:
    batch_item_spin
    Where each keyword means the following
    batch: the batch production of the CBC catalog, 
           where all the CBC items are children of
    item: the individual CBC within a batch, usually 
          ranging from 0-300ish
    spin: 0/1, indicating spins misaligned (0), or 
          spins aligned (1)
    """
    return "{}_{}_{}".format(batch,spin,item)

def makeOutDir(base,spec):
    return os.path.join(base,spec)

def makePriorFilePath(typ,spin):
    if typ not in ("BBH","NSBH"):
        raise ValueError("{} is not one of supported types: ('BBH','NSBH')".format(typ))
        return -1
    if spin not in (1,0):
        raise ValueError("{} is not one of supported spins: (0,1)".format(spin))
        return -1

    if typ=="BBH":
        if spin==1: # BBH, spins aligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/bbh_aligned_gwtc.prior"
        else: # BBH, spins misaligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/bbh_precessing_gwtc.prior"
    else:
        if spin==1: # NSBH, spins aligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/nsbh_aligned_gwtc.prior"
        else: # NSBH, spins misaligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/nsbh_precessing_gwtc.prior"

def getSpinFromName(name):
    if name=='aligned':
        return 1
    elif name=='precessing':
        return 0
    else:
        print("Something went wrong in getSpinFromName function, name supplied: {}".format(name))

def selectSpin():
    return np.random.randint(0,2)
    
def makeInjection_dict(injKeys,row):
    myDict = {}
    for k in injKeys:
        # if k in ("chi_1", "chi_2"): # handling the tilt/chi issues
        #     myDict[k] = row["tilt_{}".format(k[-1])]
        if k in ["ra"]:
            myDict[k] = np.deg2rad(row["m"+k]) # SkySim is over -180 to 180 deg, while Bilby defaults to 0 to 2pi
        elif k in ["dec"]:
            myDict[k] = np.deg2rad(row["m"+k]) # Skysim is from -90 to 90 deg, while bilby is -pi/2 to pi/2
        else:
            myDict[k] = row[k]
    myDict["geocent_time"] = ut.fakeGeoCentTime()
    for ent in myDict.keys():
        myDict[ent] = float(myDict[ent])
    return myDict

def getInjection_keys(typ,spin,ppath):
    if typ not in ("BBH","NSBH"):
        raise ValueError("{} is not one of supported types: ('BBH','NSBH')".format(typ))
        return -1
    if spin not in (1,0):
        raise ValueError("{} is not one of supported spins: (0,1)".format(spin))
        return -1
    return bb.gw.prior.CBCPriorDict(ppath).sample().keys()


def getInjectionFilePath(startPath,label):
    return os.path.join(startPath,label)+"_injection.json"

## Order
- makeLabel
- outDirectory
- prior file path
- injection dictionary

In [4]:
basePath = "/pscratch/sd/s/seanmacb/proj411ProdRuns/catalogs"
CBCType = ["BBH"]
alignment = ["precessing","aligned"]
subDir = ["Uniform,r","Uniform,u","Uniform,y","Uniform,StellarMass","Uniform,Uniform"]

In [4]:
dataDir="/global/u1/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs"
# pd.read_csv()
msk = [(x.endswith("withSNRs.csv") and (x.__contains__("BBH"))) for x in os.listdir(dataDir)] 
# To include BBH's, add an or statement to the second conditional
files = np.sort(np.array(os.listdir(dataDir))[msk])

In [5]:
files

array(['UniformParent,StellarMassWeight,BBH,aligned,withSNRs.csv',
       'UniformParent,StellarMassWeight,BBH,precessing,withSNRs.csv',
       'UniformParent,UniformWeight,BBH,aligned,withSNRs.csv',
       'UniformParent,UniformWeight,BBH,precessing,withSNRs.csv',
       'UniformParent,rWeight,BBH,aligned,withSNRs.csv',
       'UniformParent,rWeight,BBH,precessing,withSNRs.csv',
       'UniformParent,uWeight,BBH,aligned,withSNRs.csv',
       'UniformParent,uWeight,BBH,precessing,withSNRs.csv',
       'UniformParent,yWeight,BBH,aligned,withSNRs.csv',
       'UniformParent,yWeight,BBH,precessing,withSNRs.csv'], dtype='<U65')

In [7]:
subsampledDF = pd.read_csv(os.path.join(dataDir,files[0]))

In [8]:
subsampledDF

,mass_ratio,chirp_mass,luminosity_distance,dec,ra,theta_jn,psi,phase,chi_1,chi_2,geocent_time,Minimum Individual SNR,Network SNR
0,0.724896,78.802388,6748.116241,65.998801,171.152778,0.534730,2.334363,3.090994,-0.175829,0.165217,1.717962e+09,4.414548,11.822045
1,0.671580,97.961446,5874.771093,39.905024,318.790600,2.429080,1.405190,3.611765,0.720904,0.058719,1.719521e+09,2.810864,16.789325
2,0.641104,91.557577,1819.203557,-3.543272,188.154332,1.951186,2.509988,6.017879,0.852209,0.004694,1.721050e+09,13.468650,27.416766
3,0.757831,84.486743,1047.164642,20.840225,256.157249,2.012726,1.480912,5.267069,-0.187839,0.044985,1.715963e+09,21.739922,43.500357
4,0.624764,146.513702,4746.784021,15.605663,39.317944,0.978953,2.212105,0.259608,0.000402,0.651600,1.719423e+09,4.357017,16.016303
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0.536847,125.636679,7477.507440,37.294363,113.251872,0.431115,0.014544,5.158970,-0.028860,-0.002938,1.719960e+09,4.262622,14.809756
149,0.596142,141.322769,4628.755328,54.358241,272.880184,0.970653,1.812012,0.371319,0.292869,0.060120,1.714748e+09,3.903811,10.410401
150,0.781835,135.829255,4276.434361,-55.676350,113.068790,0.714991,0.977441,1.339576,0.328334,0.633143,1.715522e+09,9.425446,31.861161
151,0.781960,145.763472,3233.375444,19.850910,234.163771,1.686072,0.732466,1.250844,-0.047894,0.088451,1.739787e+09,4.648750,11.101480


In [11]:
toWrite = np.array([])
labels_snrs_dict = {}
fieldKeys = ["label","outdir","prior-file", "injection-dict","injection-file"]

for f in files:
    subsampledDF = pd.read_csv(os.path.join(dataDir,f))
    parentWeighting,cbcWeighting,CBCType,alignment,__ = f.split(",")
    
    
# for b,alignment in zip(batchPaths,alignmentArr):
    # batchDF = pd.read_csv(os.path.join(os.getcwd(),catalogPath,b))
    batchPath = "/pscratch/sd/s/seanmacb/proj411ProdRuns/catalogs/{},{}".format(parentWeighting[:-6],cbcWeighting[:-6])
    os.makedirs(batchPath,exist_ok=True)

    for index,row in subsampledDF.iterrows():
        fullItem = "{}_{}".format(CBCType,index)
        # spin=alignment # Choose spins to be aligned (1) or misaligned (0)
        label = makeLabel(parentWeighting+","+cbcWeighting,fullItem,alignment)
        outDir = makeOutDir(batchPath,label)
        priorPath = makePriorFilePath(CBCType,getSpinFromName(alignment))
        injKeys = getInjection_keys(CBCType,getSpinFromName(alignment),priorPath)
        injDict = makeInjection_dict(injKeys,row)
        injectionFPath = getInjectionFilePath(batchPath,label)

        labels_snrs_dict[label] = row["Network SNR"]

        toWrite = np.append(toWrite,[label,outDir,priorPath,injDict,injectionFPath])

    # Reshape the final array
    toWrite = np.reshape(toWrite,(-1,4))

    # Create the .ini 
    ini_basepath = os.path.join(basePath,"iniFiles")
    if not os.path.isdir(ini_basepath):
        os.mkdir(ini_basepath)

    # Writing all the .ini files
    for writeables in toWrite:
        with open("{}_base.ini".format(CBCType), "r") as f:
            read = f.readlines()
    
        writeFile = os.path.join(ini_basepath,writeables[0])+".ini"
        
        for field,value in zip(fieldKeys,writeables):
            read.append("\n")
            read.append("{}={}\n".format(field,value))
    
        with open(writeFile, "w") as f:
            f.writelines(read)

    # Write the injection file
    injectionFilePath = os.path.join(batchPath,"injectionFiles")
    os.makedirs(injectionFilePath,exist_ok=True)
    for injectionDictionary,out in zip(toWrite[:,3],toWrite[:,1]):
        with open(os.path.join(injectionFilePath,"{}_injection.json".format(label)), "w") as f:
            json.dump(injectionDictionary,f)

    # labels_snrs_dict
    sortedDict = {k: v for k, v in sorted(labels_snrs_dict.items(), key=lambda item: item[1])}
    result = [k for k in list(sortedDict.keys())[9::10]]
    print("For {}".format(f))
    for r in result:
        print(r,sortedDict[r])
    print("Finished writing for {}".format(f),end="\n\n\n")

NameError: name 'bb' is not defined